In [1]:
import numpy as np

# Set Up Environment
from kaggle_environments.envs.halite.helpers import *
from kaggle_environments import evaluate, make

import basic_bot
import task_force_bot
# import bot_swarm

In [2]:
board_size = 13
environment = make("halite", configuration={"size": board_size, "startingHalite": 5000})
agent_count = 4
environment.reset(agent_count)
state = environment.state[0]

obs = state.observation
config = environment.configuration

board = Board(state.observation, environment.configuration)
print(board)

| 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 |
| 1 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 4 | 1 |
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 0 | 0 | 0 |a0 | 1 | 0 | 1 | 0 | 1 |b0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 3 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 |
| 0 | 0 | 0 |c0 | 1 | 0 | 1 | 0 | 1 |d0 | 0 | 0 | 0 |
| 0 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 1 | 0 | 0 | 0 | 0 |
| 1 | 4 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 4 | 1 |
| 0 | 1 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 0 | 1 | 0 |



In [8]:
board_halite = basic_bot.board_halite_(obs, config)
board_ships = basic_bot.board_ships_(obs, config)
board_shipyards = basic_bot.board_shipyards_(obs, config)

distance_matrix = basic_bot.create_distance_matrix(config['size'])

In [9]:
shipyard_pos_1 = np.nan
shipyard_pos_2 = np.nan

starting_position = 42
shipyards_min_distance = 4

In [10]:
task_force_bot.agent(obs, config)

nan
nan
nan


{}

In [295]:
def score(board_halite: np.ndarray, board_shipyards: np.ndarray, ship: list,
          player_id: int, size: int, distance_matrix: np.ndarray) -> np.ndarray:
    halite_per_turn = np.zeros((size, size))
    
    ship_position = ship[0]
    ship_halite = ship[1]
    
    shipyard_counts = np.column_stack(np.unique(board_shipyards[~np.isnan(board_shipyards)], return_counts=True))
    shipyard_coordinates_array = np.column_stack(np.where(board_shipyards == player_id))
    
    if shipyard_counts[shipyard_counts[:, 0] == player_id].size == 0:  # Player has no shipyard
        halite_per_turn += 0.25 * board_halite / (1 + 2 * distance_matrix[ship_position].reshape(size, -1))
    
    else:  # Player has at least one shipyard
        distance_matrix_to_closest_shipyard = np.empty((size, size))
        distance_matrix_to_closest_shipyard[:] = np.nan
        
        for shipyard_coordinates in shipyard_coordinates_array:
            shipyard_position = basic_bot.get_position(shipyard_coordinates, size)
            distance_matrix_to_closest_shipyard = np.fmin(distance_matrix_to_closest_shipyard,
                                                          distance_matrix[shipyard_position].reshape(size, -1))
            
            halite_per_turn[shipyard_coordinates[0], shipyard_coordinates[1]] = ship_halite / (distance_matrix[ship_position, shipyard_position] + 1)
        
        halite_per_turn += 0.25 * board_halite / (1 + distance_matrix[ship_position].reshape(size, -1)
                                                  + distance_matrix_to_closest_shipyard)
    return halite_per_turn.round(1)

In [296]:
player_id = 1
ship = [0, 100]
size = 5
distance_matrix = basic_bot.create_distance_matrix(size)

print(board_halite)
print(score(board_halite, board_shipyards, ship, player_id, size, distance_matrix))

[[  0  19   0  19   0]
 [  0 209   1 209   0]
 [  0  20   0  20   0]
 [  0 209   1 209   0]
 [  0  19   0  19   0]]
[[ 0.  52.4  0.  34.9  0. ]
 [ 0.  13.1  0.  10.4  0. ]
 [ 0.   0.8  0.   0.7  0. ]
 [ 0.   8.7  0.   7.5  0. ]
 [ 0.   1.2  0.   1.   0. ]]


In [95]:
ship_score = score(board_halite, board_shipyards, ship, player_id, size, distance_matrix)

In [233]:
ship_score

array([[ 0. , 52.4,  0. , 34.9,  0. ],
       [ 0. , 13.1,  0. , 10.4,  0. ],
       [ 0. ,  0.8,  0. ,  0.7,  0. ],
       [ 0. ,  8.7,  0. ,  7.5,  0. ],
       [ 0. ,  1.2,  0. ,  1. ,  0. ]])

In [297]:
target_coordinates = np.array(np.unravel_index(np.argmax(ship_score), ship_score.shape))
target_position = basic_bot.get_position(target_coordinates, config['size'])

In [298]:
print(target_coordinates)
print(target_position)

[0 1]
1


In [29]:
board_shipyards[0, 1] = 1
board_shipyards[0, 3] = 1
board_shipyards[1, 3] = 2
board_shipyards[3, 1] = 3
board_shipyards[3, 3] = 4

In [101]:
ship_dict = {'0-1': [6, 10], '1-1': [8, 16], '2-1': [13, 3], '3-1': [1, 5], '4-1': [15, 8]}

In [106]:
{key: value for key, value in sorted(ship_dict.items(), key=lambda item: item[1][1], reverse=True)}

{'1-1': [8, 16], '0-1': [6, 10], '4-1': [15, 8], '3-1': [1, 5], '2-1': [13, 3]}

In [104]:
ship_dict.items()

dict_items([('0-1', [6, 10]), ('1-1', [8, 16]), ('2-1', [13, 3]), ('3-1', [1, 5]), ('4-1', [15, 8])])

In [107]:
board_shipyards

array([[nan,  1., nan,  1., nan],
       [nan, nan, nan,  2., nan],
       [nan, nan, nan, nan, nan],
       [nan,  3., nan,  4., nan],
       [nan, nan, nan, nan, nan]])

In [108]:
board_ships

array([[nan, nan, nan, nan, nan],
       [nan,  0., nan,  1., nan],
       [nan, nan, nan, nan, nan],
       [nan,  2., nan,  3., nan],
       [nan, nan, nan, nan, nan]])

In [109]:
board_halite

array([[  0,  19,   0,  19,   0],
       [  0, 209,   1, 209,   0],
       [  0,  20,   0,  20,   0],
       [  0, 209,   1, 209,   0],
       [  0,  19,   0,  19,   0]])

In [7]:
player_id = obs['player']

ships_dict = obs['players'][player_id][2]

for ship in ships_dict:
    ship_position = ships_dict[ship][0]
    ship_score = board_halite / (2 * distance_matrix[ship_position].reshape(config['size'], -1) + 1)

In [284]:
current_position = 4
target_position = 6
size = 5
current_coordinates = basic_bot.get_coordinates(current_position, size)
target_coordinates = basic_bot.get_coordinates(target_position, size)
blocked_squares = np.zeros((config['size'], config['size']), dtype=bool)
distance_matrix = basic_bot.create_distance_matrix(size)

In [273]:
def pathfinder(current_position: int, target_position: int, blocked_squares: np.ndarray,
               distance_matrix: np.ndarray, size: int) -> Union[str, None]:
    
    current_coordinates = basic_bot.get_coordinates(current_position, size)
    target_coordinates = basic_bot.get_coordinates(target_position, size)
    
    directions_dict = {'NORTH': [-1, 0], 'EAST': [0, 1], 'SOUTH': [1, 0], 'WEST': [0, -1], 'None': [0, 0]}
    
    directions_values = {}
    for direction in directions_dict:
        directions_values[direction] = (distance_matrix[target_position].reshape(size, -1)
                                           .item(tuple(current_coordinates + directions_dict[direction]))
                                        - distance_matrix[target_position].reshape(size, -1)
                                              .item(tuple(current_coordinates)))
    
    for direction in directions_dict:
        if blocked_squares.item(tuple(current_coordinates + directions_dict[direction])):
            directions_values.pop(direction, None)
    
    directions_values_sorted = {key: value for key, value in sorted(directions_values.items(), key=lambda item: item[1])}
    
    print(directions_values_sorted)
    
    # For the rare occasion that all field are blocked use try and stay put if nothing is possible
    try:
        return list(directions_values_sorted)[0]
    except IndexError:
        return 'None'

In [274]:
pathfinder(current_position, target_position, blocked_squares, distance_matrix, size)

{'None': 0, 'NORTH': 1, 'EAST': 1, 'SOUTH': 1, 'WEST': 1}


'None'

In [3]:
env = make('halite', configuration={'episodeSteps': 3, 'size':21}, debug=True)
env.configuration

{'episodeSteps': 3,
 'size': 21,
 'agentTimeout': 30,
 'actTimeout': 6,
 'runTimeout': 9600,
 'isProduction': False,
 'startingHalite': 24000,
 'spawnCost': 500,
 'convertCost': 500,
 'moveCost': 0,
 'collectRate': 0.25,
 'regenRate': 0.02,
 'maxCellHalite': 500,
 'randomSeed': 1997679080}

In [4]:
env.run(["task_force_bot.py", "basic_bot.py", "basic_bot.py", "basic_bot.py"])
env.render(mode="ipython", width=800, height=600)

nan
nan
nan
[[ 94.86 285.6    0.     0.    53.04   0.     7.14 291.72  40.8    0.
    0.     0.    40.8  291.72   7.14   0.    53.04   0.     0.   285.6
   94.86]
 [103.02   0.     0.     0.     0.     0.     0.     0.   180.54   0.
    0.     0.   180.54   0.     0.     0.     0.     0.     0.     0.
  103.02]
 [ 82.62 219.3    0.   339.66   0.     0.     0.     0.     0.     0.
    0.     0.     0.     0.     0.     0.     0.   339.66   0.   219.3
   82.62]
 [  0.     0.     0.   215.22   0.     0.     0.    65.28   0.   206.04
   54.06 206.04   0.    65.28   0.     0.     0.   215.22   0.     0.
    0.  ]
 [133.62  24.48   0.   131.58   0.    68.34   0.     0.     0.    38.76
    0.    38.76   0.     0.     0.    68.34   0.   131.58   0.    24.48
  133.62]
 [  0.     0.     0.     0.     0.   304.     0.   196.86   0.    84.66
    0.    84.66   0.   196.86   0.     0.     0.     0.     0.     0.
    0.  ]
 [177.48  49.98   0.     0.     0.     0.    68.34  91.8    0.     0.
   40.8 